<a href="https://colab.research.google.com/github/SEEsuite/colab_scripts/blob/main/english_5_star_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[model](https://huggingface.co/LiYuan/amazon-review-sentiment-analysis)

I beg you, do not use this for creative purposes - use it for product reviews at scale. A similar model is infamous (in AI community) for a sexist resume reader that Amazon was experimenting with. It simply learned that amazon HR unfairly rates women worse and carried that pattern forward for new candidates.


The model has an accuracy of 80% - this is a hard problem and again, is useful for rating products at scale. 

In [1]:
### HERE IS THE CELL YOU NEED TO CHANGE
## DONT USE TWITTER DATASET
link = "https://docs.google.com/spreadsheets/d/1m1-qV00Qkm2m9Znypj_ORBZgAQ9yQ9eO/edit?usp=sharing&ouid=101042095541764641159&rtpof=true&sd=true"
### YOU WILL NEED TO CHANGE THE EXCEL NAMES OR THE DF NAMES BELOW TO MATCH

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.3 MB/s eta 0:00:00


In [3]:
# huggingface's tools for pretrained language models
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer

In [4]:
# importing miscelaneaous packages 
import numpy as np # fast manipulation of multidimensional arrays

from tqdm.notebook import tqdm as progress_bar # a little vizualization of how fast a loop is running
from scipy.special import softmax

In [5]:
import urllib.request
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

def import_data_from_drive(share_link, your_name_for_file="my_data"):
  """Brings data file from a google drive sharepoint to your colab workspace.
     It does not require you to host the dataset on your own account.

     Parameters:
     share_link: the link to view a file in google drive
     our_name_for_file: a string describing the file, preferable endling in a file type, ex. 'data.csv'
     """
  id = share_link.split("/")[5] # separate the id from the link
  print("Using id", id, "to find file on drive")

  # use pydrive and colab modules to authenticate you
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)
  print("Authenticated colab user")

  # This step will move the file from Drive to the workspace
  download_object = drive.CreateFile({'id':id}) 
  download_object.GetContentFile(your_name_for_file)
  print("Added file to workspace with name", your_name_for_file)

  return

### Load Data to DataFrame
We will load some tweets from a shared google drive file, which will require us to sign into our google drive accounts. 
[backup link to shared file](https://docs.google.com/spreadsheets/d/1m1-qV00Qkm2m9Znypj_ORBZgAQ9yQ9eO/edit?usp=sharing&ouid=101042095541764641159&rtpof=true&sd=true)

<!-- # id = '1prY6_BgwYrUHCSJ0DYAB4KA7HcjTdDtU' -->

*key python package: pandas*   
[pandas documentation](https://pandas.pydata.org/docs/getting_started/index.html#getting-started). 

[visual cheatsheet](https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf)

In [6]:
import pandas as pd # basically the excel of python

In [7]:

import_data_from_drive(link, your_name_for_file="tweets.xlsx")
df = pd.read_excel('tweets.xlsx')
# df = pd.read_csv('tweets.xlsx')

Using id 1m1-qV00Qkm2m9Znypj_ORBZgAQ9yQ9eO to find file on drive
Authenticated colab user
Added file to workspace with name tweets.xlsx


In [8]:

# df is now an object, with associated methods we can use
df.head(5) # lets look at the first five data samples
# you can even access the spreadsheet in colab... 

,Date,Full Text,Clean Text,Author,Url,Continent,Country,Region,Country Code,Continent Code,Region Code,City Code,Twitter Followers,Twitter Following,Twitter Reply Count,Twitter Retweets,Twitter Verified,Reach (new)
0,2022-10-01 23:40:00.000,"In Colorado Senate race, Michael Bennet still ...",in colorado senate race michael bennet still f...,Prison_Health,http://twitter.com/Prison_Health/statuses/1576...,North America,United States of America,Hawaii,USA,NORTH AMERICA,USA.HI,USA.HI.Honolulu,19711,2715,0,0,False,7325
1,2022-10-01 23:27:28.000,COMING UP on @WRAL at 7:30pm: We sit down with...,coming up on at 730pm we sit down with and abo...,BryanRAnderson,http://twitter.com/BryanRAnderson/statuses/157...,North America,United States of America,North Carolina,USA,NORTH AMERICA,USA.NC,USA.NC.Raleigh,3832,1103,2,4,True,13263
2,2022-10-01 23:16:38.000,Summaries of high-profile Supreme Court cases:...,summaries of highprofile supreme court cases t...,January20th49,http://twitter.com/January20th49/statuses/1576...,North America,United States of America,Ohio,USA,NORTH AMERICA,USA.OH,NaN,39,300,0,1,False,0
3,2022-10-01 23:05:12.000,Abortion Icon Emma Bonino Trounced in Italian ...,abortion icon emma bonino trounced in italian ...,UsBurning,http://twitter.com/UsBurning/statuses/15763475...,North America,United States of America,Georgia,USA,NORTH AMERICA,USA.GA,USA.GA.Atlanta,360,34,0,0,False,0
4,2022-10-01 22:02:12.000,💥38 DAYS UNTIL #ELECTIONDAY MIDTERMS💥 WHAT R U...,💥38 days until midterms💥 what r u doing for de...,LeviFetterman,http://twitter.com/LeviFetterman/statuses/1576...,North America,United States of America,Pennsylvania,USA,NORTH AMERICA,USA.PA,NaN,33774,1702,1,10,False,16039


## Clean Data
Gameplan: define a function that cleans one tweet. Then apply this function to every tweet in our dataframe.

Regex is a shorthand for writing logical statements to match strings - we are just copying and pasting the regex today, its not important to learn all the shorthand yet. But if you are interested:

[using regex to remove hashtags](https://catriscode.com/2021/03/02/extracting-or-removing-mentions-and-hashtags-in-tweets-using-python/)   
[regex cheatsheet](https://www.interviewbit.com/regex-cheat-sheet/)


*key python package: regex*

In [9]:
import re # search through and clean text

In [10]:
print(df['Full Text'][0])

In Colorado Senate race, Michael Bennet still fights for child tax credit and immigration reform https://t.co/F5ak34HrCE


In [11]:
# running this cell defines the function, does not run the function

def clean(tweet):

  # remove uppercase letters
  tweet = tweet.lower()

  # remove mentions
  tweet = re.sub("@[A-Za-z0-9_]+", "", tweet)
  # remove hashtags
  tweet = re.sub("#[A-Za-z0-9_]+", "", tweet)
  # remove links
  tweet = re.sub(r"http\S+", "", tweet)

  return tweet

In [12]:
df['Clean Text'] = df['Full Text'].apply(clean)

In [13]:
print(df['Clean Text'][0])

in colorado senate race, michael bennet still fights for child tax credit and immigration reform 


In [14]:
!rm -r cardiffnlp # just in case

rm: cannot remove 'cardiffnlp': No such file or directory


In [15]:
# MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest" 

from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("LiYuan/amazon-review-sentiment-analysis")

model = AutoModelForSequenceClassification.from_pretrained("LiYuan/amazon-review-sentiment-analysis")

labels = ["1 star", "2 stars", "3 stars","4 stars", "5 stars"]

In [16]:
# deep learning toolkit
from torch.utils.data import DataLoader
from torch.nn import Softmax
import torch

In [17]:
CUDA = torch.cuda.is_available()
print("Using gpu:", CUDA)
processor = 'cuda'

Using gpu: True


In [18]:
# making some pytorch variables to assist us
batch_size = 32
dl = DataLoader(df['Clean Text'], batch_size=batch_size) # seems arbitrary now but becomes useful once we want to control more factors of the dataset.
print("Number of Batches", len(dl))

Number of Batches 592


In [19]:
# lets run it! It should take about a minute or so

# this line makes an empty dataframe to hold the scores we are about to calculate
output_df = pd.DataFrame()

# this moves the model to a out chosen processor
model = model.to(processor)

for text_batch in progress_bar(dl): # we loop through every batch in our dataset
  # print(text_batch)
  encoded_input = tokenizer(text_batch, return_tensors='pt', padding=True, truncation=True) # turn each tweet string into tokens
  encoded_input.to(processor) # input must be on the same processor as the model

  output = model(**encoded_input) # apply the model

  embeddings = output[0].detach().cpu().numpy() # gross rearrangment of the data type, a messy intermediate step

  # append new embeddings to current dataframe through pandas manipulations
  embed_df = pd.DataFrame(embeddings, columns=labels)
  # print(embed_df)
  output_df = pd.concat((output_df, embed_df), axis=0, ignore_index=True)

  0%|          | 0/592 [00:00<?, ?it/s]

In [20]:
# convert embeddings (vectors) to a probability distribution
sentiment_df = output_df.apply(softmax, axis=1, result_type='broadcast') 
sentiment_df
# all rows should sum to one

,1 star,2 stars,3 stars,4 stars,5 stars
0,0.089491,0.063757,0.113751,0.263241,0.469760
1,0.013546,0.010575,0.025797,0.177615,0.772467
2,0.065803,0.043621,0.088150,0.226366,0.576061
3,0.264026,0.088513,0.098445,0.155265,0.393751
4,0.107748,0.037677,0.047646,0.157275,0.649652
...,...,...,...,...,...
18912,0.843235,0.094175,0.023498,0.011591,0.027502
18913,0.150209,0.060138,0.077030,0.173195,0.539428
18914,0.306938,0.128033,0.108283,0.134115,0.322632
18915,0.678943,0.197049,0.076841,0.020251,0.026916


In [21]:
# Transform scores to labels for easier analysis
def get_class(row):
  max_idx = np.argmax(row)
  return labels[max_idx]

sentiment_df["predicted_rating"] = sentiment_df.apply(get_class, axis=1)

In [22]:
final_df = pd.concat((df, sentiment_df), axis=1)

In [23]:
final_df.head()

,Date,Full Text,Clean Text,Author,Url,Continent,Country,Region,Country Code,Continent Code,...,Twitter Reply Count,Twitter Retweets,Twitter Verified,Reach (new),1 star,2 stars,3 stars,4 stars,5 stars,predicted_rating
0,2022-10-01 23:40:00.000,"In Colorado Senate race, Michael Bennet still ...","in colorado senate race, michael bennet still ...",Prison_Health,http://twitter.com/Prison_Health/statuses/1576...,North America,United States of America,Hawaii,USA,NORTH AMERICA,...,0,0,False,7325,0.089491,0.063757,0.113751,0.263241,0.469760,5 stars
1,2022-10-01 23:27:28.000,COMING UP on @WRAL at 7:30pm: We sit down with...,coming up on at 7:30pm: we sit down with and...,BryanRAnderson,http://twitter.com/BryanRAnderson/statuses/157...,North America,United States of America,North Carolina,USA,NORTH AMERICA,...,2,4,True,13263,0.013546,0.010575,0.025797,0.177615,0.772467,5 stars
2,2022-10-01 23:16:38.000,Summaries of high-profile Supreme Court cases:...,summaries of high-profile supreme court cases:...,January20th49,http://twitter.com/January20th49/statuses/1576...,North America,United States of America,Ohio,USA,NORTH AMERICA,...,0,1,False,0,0.065803,0.043621,0.088150,0.226366,0.576061,5 stars
3,2022-10-01 23:05:12.000,Abortion Icon Emma Bonino Trounced in Italian ...,abortion icon emma bonino trounced in italian ...,UsBurning,http://twitter.com/UsBurning/statuses/15763475...,North America,United States of America,Georgia,USA,NORTH AMERICA,...,0,0,False,0,0.264026,0.088513,0.098445,0.155265,0.393751,5 stars
4,2022-10-01 22:02:12.000,💥38 DAYS UNTIL #ELECTIONDAY MIDTERMS💥 WHAT R U...,💥38 days until midterms💥 what r u doing for d...,LeviFetterman,http://twitter.com/LeviFetterman/statuses/1576...,North America,United States of America,Pennsylvania,USA,NORTH AMERICA,...,1,10,False,16039,0.107748,0.037677,0.047646,0.157275,0.649652,5 stars


## Save your work

In [24]:
# let's save it
# this will only save to worksplace and then you have to download it to your local computer or upload to drive
save_path = "predicted_rating_scores.xlsx"
final_df.to_excel(save_path)